In [1]:
# Import the required libraries
import pandas as pd
import networkx as nx

In [2]:
edges_df = pd.read_csv('../data/gephi-edge-list.csv')
nodes_df = pd.read_csv('../data/gephi-node-list.csv')

# Ensure ID columns are clean
nodes_df['ID'] = nodes_df['ID'].astype(str).str.strip()
edges_df['Source'] = edges_df['Source'].astype(str).str.strip()
edges_df['Target'] = edges_df['Target'].astype(str).str.strip()

# Create a mapping from player name to node ID
player_to_id = dict(zip(nodes_df['name'], nodes_df['ID']))


In [3]:
# Initialize graph
G = nx.Graph()

# Add nodes with names as attributes
for _, row in nodes_df.iterrows():
    G.add_node(row['ID'], name=row['name'],college_conference=row['college_conference'])

# Add edges with weights
for _, row in edges_df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

In [4]:
# Compute sum of neighbor edge weights (and use names in output)
sum_neighbor_weights = {}

for node_id in G.nodes():
    neighbors = list(G.neighbors(node_id))
    seen_edges = set()  # To avoid double-counting undirected edges
    total = 0
    
    for neighbor in neighbors:
        for nbr_of_neighbor in G.neighbors(neighbor):
            edge = tuple(sorted([neighbor, nbr_of_neighbor]))
            if edge not in seen_edges:
                total += G[neighbor][nbr_of_neighbor]['weight']
                seen_edges.add(edge)
    
    node_name = G.nodes[node_id]['name']
    sum_neighbor_weights[node_name] = total

# Convert to DataFrame
df = pd.DataFrame(sum_neighbor_weights.items(), columns=["Node", "Sum of Neighbour's Edge Weights"])
print(df)

                Node  Sum of Neighbour's Edge Weights
0     Adam Vinatieri                     48262.125519
1       Jason Peters                         0.000000
2     Terence Newman                     24527.738000
3      DeAngelo Hall                      6889.435667
4       Robbie Gould                      8893.807460
...              ...                              ...
1951   Michael Penix                     75897.802000
1952  Caleb Williams                     95249.109286
1953     Rome Odunze                     95249.109286
1954  Malachi Corley                    101174.800154
1955     Trey Benson                     71622.545000

[1956 rows x 2 columns]


In [5]:
for node_id in G.nodes():
    neighbors = list(G.neighbors(node_id))
    seen_edges = set()  # To avoid double-counting undirected edges
    total = 0
    
    for neighbor in neighbors:
        for nbr_of_neighbor in G.neighbors(neighbor):
            edge = tuple(sorted([neighbor, nbr_of_neighbor]))
            if edge not in seen_edges:
                total += G[neighbor][nbr_of_neighbor]['weight']
                seen_edges.add(edge)
    
    node_name = node_id  # Use node_id directly
    sum_neighbor_weights[node_name] = total

# Convert to DataFrame
sum_neighbor_df = pd.DataFrame(sum_neighbor_weights.items(), columns=["ID", "Sum of Neighbour's Edge Weights"])

# Merge sum_neighbor_df with nodes_df to get college_conference info
final_df = pd.merge(sum_neighbor_df, nodes_df[['ID', 'college_conference']], on='ID', how='inner')

# Group by college_conference and sum the weights
conference_sum = final_df.groupby('college_conference')['Sum of Neighbour\'s Edge Weights'].sum().reset_index()


In [6]:
# Round the 'Sum of Neighbour's Edge Weights' column to 2 decimal places for cleaner display
conference_sum['Sum of Neighbour\'s Edge Weights'] = conference_sum['Sum of Neighbour\'s Edge Weights'].round(2)

# Sort the DataFrame by 'Sum of Neighbour's Edge Weights' in descending order
conference_sum_sorted = conference_sum.sort_values(by='Sum of Neighbour\'s Edge Weights', ascending=False)
conference_sum_sorted = conference_sum_sorted.reset_index(drop=True)

# Print the sorted DataFrame
print(conference_sum_sorted)



                                  college_conference  \
0                            Southeastern Conference   
1                                 Big Ten Conference   
2                          Atlantic Coast Conference   
3                          Pacific Twelve Conference   
4                              Big Twelve Conference   
5                       American Athletic Conference   
6                                     Conference USA   
7                           Mountain West Conference   
8                                        Independent   
9                            Mid-American Conference   
10                               Sun Belt Conference   
11               Missouri Valley Football Conference   
12                                          Big East   
13                            Pacific Ten Conference   
14                                Big Sky Conference   
15                                        Ivy League   
16                     Colonial Athletic Associa

In [12]:
# Calculate mean of neighbors' edge weights per player
mean_neighbor_weights = []

for node_id in G.nodes:
    neighbors = list(G.neighbors(node_id))
    weights = []

    for neighbor in neighbors:
        for nbr_of_neighbor in G.neighbors(neighbor):
            if (neighbor, nbr_of_neighbor) in G.edges:
                weights.append(G[neighbor][nbr_of_neighbor]['weight'])

    node_name = G.nodes[node_id]['name']
    mean_weight = sum(weights) / len(weights) if weights else 0
    mean_neighbor_weights.append({
        'ID': node_id,
        'name': node_name,
        'mean_neighbor_edge_weight': mean_weight
    })

# Merge with nodes_df to get conference info
mean_df = pd.DataFrame(mean_neighbor_weights)
merged_df = pd.merge(mean_df, nodes_df[['ID', 'college_conference']], on='ID', how='left')

# Group by conference and compute mean of means
conference_mean = merged_df.groupby('college_conference')['mean_neighbor_edge_weight'].mean().reset_index()

# Sort into descending order
conference_mean_sorted = conference_mean.sort_values(by='mean_neighbor_edge_weight', ascending=False)

# Save to CSV in same directory as node and edge .csv files
conference_mean_sorted.to_csv('../data/conference_teammate_mean_fantasy_points.csv', index=False)


print(conference_mean_sorted.head())

                              college_conference  mean_neighbor_edge_weight
23               Mid-States Football Association                 132.261676
4             Big South - Ohio Valley Conference                 131.578780
51                   Western Athletic Conference                 129.788326
30                New Jersey Athletic Conference                 128.850535
47  Southern Intercollegiate Athletic Conference                 121.177387
